In [63]:
import requests
import bs4
from tqdm import tqdm
import pandas as pd
import numpy as np
import re
# 爬ptt政治板
def get_some_page_ptt_data(URL,last_n_page):
    def get_content(soup):
        ## 查找所有html 元素 抓出內容
        main_container = soup.find(id='main-container')
        # 把所有文字都抓出來
        all_text = main_container.text
        # 把整個內容切割透過 "-- " 切割成2個陣列
        pre_text = all_text.split('--')[0]
        # 把每段文字 根據 '\n' 切開
        texts = pre_text.split('\n')
        # 如果你爬多篇你會發現 
        contents = texts[2:]
        # 內容
        content = ''.join(contents)
        return content
    
    # 根據PTT的URL取得這個PTT頁面的資料的DataFrame格式
    def get_this_index_data(URL):
        
        # 發送get 請求 到 ptt 八卦版
        response = requests.get(URL, headers = {'cookie': 'over18=1;'})
        
        #  把網頁程式碼(HTML) 丟入 bs4模組分析
        soup = bs4.BeautifulSoup(response.text,"html.parser")

        ## PTT 上方4個欄位
        result = soup.find_all('div','title')
        urls = []
        titles = []
        contents = []
        for i in result:
            try:
                post_url = 'https://www.ptt.cc'+str(i).split('href="')[1].split('">')[0]
                urls.append(post_url)
                titles.append(str(i).split("]")[1].split("<")[0])
                response = requests.get(post_url, headers = my_headers)
                soup = bs4.BeautifulSoup(response.text,"html.parser")
                contents.append(get_content(soup))
            except:
                pass #(本文已被刪除)
        df = pd.DataFrame()
        df['title'] = titles
        df['url'] = urls
        df['content'] = contents
        return df
    
    # 計算目前ptt政治版總共有多少page
    def get_total_page_number(URL):
        response = requests.get(URL, headers = {'cookie': 'over18=1;'})
        soup = bs4.BeautifulSoup(response.text,"html.parser")
        number = int(str(soup.select('#action-bar-container > div > div.btn-group.btn-group-paging > a:nth-child(2)')[0]).split('index')[1].split('.html')[0])
        return number
    
    total_page_number = get_total_page_number(URL)
    
    # 取得一些政治板page的連結稱為URLS
    URLS = [f"https://www.ptt.cc/bbs/HatePolitics/index{i}.html" for i in range(total_page_number,total_page_number-last_n_page,-1)]
    
    # 根據特定URL取得這個PAGE的資料dataframe格式
    def get_this_index_data(URL):
        # 設定Header與Cookie
        my_headers = {'cookie': 'over18=1;'}
        # 發送get 請求 到 ptt 八卦版
        response = requests.get(URL, headers = my_headers)
        #  把網頁程式碼(HTML) 丟入 bs4模組分析
        soup = bs4.BeautifulSoup(response.text,"html.parser")
        ## PTT 上方4個欄位
        result = soup.find_all('div','title')
        urls = []
        titles = []
        contents = []
        for i in tqdm(result):
            try:
                titles.append(i.text.replace("\n",''))
                urls.append('https://www.ptt.cc'+str(i.find_all('a')[0]['href']))
                response = requests.get(urls[-1], headers = my_headers)
                soup = bs4.BeautifulSoup(response.text,"html.parser")
                contents.append(get_content(soup))
            except:
                print('頁面刪除')
        df = pd.DataFrame()
        min_len = min(len(titles),len(urls),len(contents))
        df['title'] = titles[:min_len]
        df['url'] = urls[:min_len]
        df['content'] = contents[:min_len]
        return df
    
    #取得PTT政治板最新的PAGE
    df = get_this_index_data(URL) 
    for u in tqdm(URLS): #對URLS做遍歷
        df = df.append(get_this_index_data(u))
    return df

In [64]:
URL = "https://www.ptt.cc/bbs/HatePolitics/index.html"
last_n_page = 10
get_some_page_ptt_data(URL,last_n_page)

100%|██████████| 20/20 [00:03<00:00,  6.03it/s]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14604\24328503.py:98: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(get_this_index_data(u))
100%|██████████| 20/20 [00:03<00:00,  5.76it/s]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14604\24328503.py:98: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(get_this_index_data(u))
 20%|██        | 2/10 [00:07<00:28,  3.61s/it]

頁面刪除


100%|██████████| 20/20 [00:03<00:00,  6.10it/s]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14604\24328503.py:98: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(get_this_index_data(u))
100%|██████████| 20/20 [00:04<00:00,  4.63it/s]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14604\24328503.py:98: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(get_this_index_data(u))
 40%|████      | 4/10 [00:15<00:23,  3.96s/it]

頁面刪除


100%|██████████| 20/20 [00:03<00:00,  5.92it/s]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14604\24328503.py:98: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(get_this_index_data(u))
100%|██████████| 20/20 [00:04<00:00,  4.26it/s]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14604\24328503.py:98: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(get_this_index_data(u))
100%|██████████| 20/20 [00:03<00:00,  5.77it/s]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14604\24328503.py:98: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(get_this_index_data(u))
100%|██████████| 20/20 [00:03<00:00,  5.56it/s]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14604\243285

,title,url,content
0,[黑特] 麥卡錫來台根本是美國人的陰謀壞透了,https://www.ptt.cc/bbs/HatePolitics/M.16753906...,麥卡錫來台根本是美國人的陰謀，美國故意要用麥卡錫來台騙解放軍開第一槍讓兩岸開戰，台灣又要變成...
1,[黑特] 小智市長真的是被欲加之罪何患無辭啊！,https://www.ptt.cc/bbs/HatePolitics/M.16753908...,我請問你各位藍白粉，你們天天在使用office套裝軟體，有誰知道軟體的原理，程式是怎麼寫出來...
2,Re: [轉錄] 翁達瑞 教育部「虛實不分」的訴願審查,https://www.ptt.cc/bbs/HatePolitics/M.16753908...,坦白講以翁教授對林智堅的標準高虹安根本沒問題引用的期刊作者是自己，只佔論文的一小部分食糞教授...
3,[討論] 小智三先為什麼答不出研究方法？,https://www.ptt.cc/bbs/HatePolitics/M.16753911...,雖然台大訴願的判決還沒出來啦但我想問的是這其中是不是有一個bug？https://i.img...
4,[新聞] 「法院就是認證陳建仁抄襲」！ 王鴻薇轟,https://www.ptt.cc/bbs/HatePolitics/M.16753911...,1.新聞網址︰※ 網址超過一行 請縮網址 ※https://www.ettoday.net/...
...,...,...,...
15,[討論] 徐巧芯不建議一下曾銘宗世代交替嗎？,https://www.ptt.cc/bbs/HatePolitics/M.16753367...,國民黨的立院黨團總召曾銘宗也已經滿64歲了按照徐巧芯的期待恐怕也該世代交替了徐巧芯要不要呼籲...
16,[討論] 綠營與政黑綠粉繼續重蹈覆徹,https://www.ptt.cc/bbs/HatePolitics/M.16753370...,郭台銘還沒入黨所以不算本來國民黨總統級候選人只有朱立倫但綠營過去四年為了防堵當時連任市長聲勢...
17,Re: [討論] 林佳龍會不會參與這場龍大祭典？,https://www.ptt.cc/bbs/HatePolitics/M.16753370...,→ TheoEpstein : 就鏈鋸人TV動畫某個ID被吹上天，結果BD銷量爛到掉渣→ ...
18,[新聞] 答不出研究方法!林智堅在審議會被考倒,https://www.ptt.cc/bbs/HatePolitics/M.16753373...,答不出論文研究方法！林智堅在審議會被考倒 挨轟「基礎能力不足」https://www.ett...
